# Packages
To run this notebook the following packages are needed:

- `DataFrames.jl`
- `Distributions.jl`
- `Plots.jl`
- `StatPlots.jl`
- `PlotlyJS.jl`

In [ ]:
#Pkg.update();
#Pkg.add("DataFrames")
#Pkg.add("Distributions")
#Pkg.add("Plots")
using DataFrames, Distributions, StatPlots;
pyplot();


# The linear model
Consider the linear model:
$$
y_i = \beta^{0}_{0} + \beta^{0}_{1} x_{i1} + \beta^{0}_{2} x_{i2} + u_i = x_i\beta^{0} + u_i \quad i=1,\ldots,n.
$$
Usually we make assumptions about the joint distribution of $\{u_i,x_i\}$. Instead we will simulate $\{u_i,x_i\}$ from different distributions. The choice of these distribution will determine the stochastic properties of ${u_i,x_i}$. This will hopefully clarify the relationship between assumptions and properties of the OLS estimator. 




## Case 1: $u_i$ and $x_i$ are independent. 

In [ ]:
using Distributions
srand(1) ## Set seed 
n = 25;  ## number of obs
β⁰ = [1.0, 0.3, 0.8]
χ² = Chisq(2);
N  = Normal(0,1)
X = [ones(n) rand(N, n) rand(N, n)];
u = (rand(χ², n)-mean(χ²))/√var(χ²);
y = X*β⁰ + u;

The code generates a vector $u$ of size $(n\times 1)$, a matrix of regressor, $X$, of dimension $n\times 3$, and a vector of dependent variables, $y$, of dimension $(n\times 1)$. The first column of $X$ is composed of ones as we are including an intercept. 

We will let $x_i$ denote the $i$-th row of $X$.

We are referring to $\beta^0$ to stress that we want to recover the value used to generate $y$. Throughout, `β⁰ = [1.0, 0.3, 0.8]`. 

Since $u$ and $x$ are simulated independently, $u_i$ and $x_i$ are indipendent, that is, $(u_i, x_i)$ is an independent sequence along $i$, but also $u_i$ and $x_i$ are independent for a given $i$. This implies that $$E[u_i | x_i] = 0.$$

In this case $\beta^0$ is the partial effect which can be consistently estimated by OLS. 


### OLS Estimator

In [ ]:

## mimick (X'X)^-1 X'Y
βhat = inv(X'X)X'y   



In [ ]:
## More efficient solve (X'X)β = X'Y
βhat = X'X\X'y

In [ ]:
## But you can simply do
βhat = X\y

As you can see, they all give the same estimate of $\beta^0$, but `X\y` is both faster and computationally more reliable. 

$\hat{\beta}_0$ and $\hat{\beta}_1$ are relatively close to $\beta^0_{0}$ and $\beta^0_{1}$, while $\hat{\beta}_2$ is not very close to $\beta^0_{2}$. But "closedness" is a probabolistic concept --- we need to take into account the sampling variation. For instance, we know that under the assumptions that we have made here, $\hat{\beta}$ is unbiased. Let see weather this is the case. 

In [ ]:
simulations = 100_000    ## This should be infinity, but 100_000 would work just fine
βhat  = Array(Float64, simulations, 3);
for j = 1:simulations
    X = [ones(n) rand(N, n) rand(N, n)];
    u = (rand(χ², n)-mean(χ²))/√var(χ²);
    y = X*β⁰ + u;
    βhat[j, 1:3] = X\y;
end
## the mean of β approximates E[\hat{β}] ≈ β₀
mean(βhat,1)

Excellent! On average the OLS estimator is equal to $\beta^0$. What about the distribution of $\hat{\beta}$?

In [ ]:
using Plots
Plots.histogram(βhat, layout = 3, normalize = true)

As suggested by the theory, the histograms of $\hat{\beta}$ tend to be normal. Let's see how good is this approximation. To this, recall what is the statement of the CLT
$$
\sqrt{n}(\hat{\beta} - \beta^0) \xrightarrow{d} N(0, \Sigma).
$$
A consistent estimator of $\Sigma$
$$
\hat{\Sigma} = \left(\frac{X'X}{n}\right)^{-1} \left(\frac{1}{n}\sum_{i=1}^n \hat{u}^2_i x_i'x_i\right) \left(\frac{X'X}{n}\right)^{-1},
$$
Since we have simulated the data, we know that $var(u_i|x_i) = \sigma^2$ and so there is not conditional heteroskedasticity. But we will play fool and we will for the moment assume that we do not know this and we  accordingly use a heteroskedastic robust variance estimator. This is not a problem because the matrix estimator above is consistent regardless of whether $u_i$ is conditional heteroskedastic or not. 

By consistency of $\hat{\Sigma}$ we have that
$$
\sqrt{n}\hat{\Sigma}^{-1/2}(\hat{\beta} - \beta^0) \xrightarrow{d} N(0, I_k),
$$
where $\hat{\Sigma}^{-1/2}$ is the Cholesky decomposition of $\hat{\Sigma}^{-1}$. The joint normality above implies that each element of the vector $\hat{beta}$ is asymptotically normal:
$$
\sqrt{n}\frac{(\hat{\beta}_j - \beta^0_j)}{\sqrt{\hat{\Sigma}_{jj}}} \xrightarrow{d} N(0, 1).
$$

Let's now code $\hat{\Sigma}$. We will write a function taking two arguments: $X$ and $\hat{u}$. We will also write a function that given y, X and $\hat{\beta}$ returns $\hat{u}$.

In [ ]:
function Sigmahat(X, uhat)
    n   = size(X,1)   ## Number of observations
    XXi = inv(X'X/n)  ## (∑xᵢ'xᵢ/n)⁻¹
    Xu  = X.*uhat     ## Xu = xᵢ ûᵢ
    XuuX= Xu'Xu/n     ## XuuX = ∑ûᵢ² xᵢ'xᵢ/n
    XXi*XuuX*XXi      ## Σ̂
end

function uh(y, X, βhat)
    y-X*βhat
end

In [ ]:
n = 25
simulations = 100_000
βhat  = Array(Float64, simulations, 3);
σ²hat  = Array(Float64, simulations, 3);
for j = 1:simulations
    X = [ones(n) rand(N, n) rand(N, n)];
    u = (rand(χ², n)-mean(χ²))/√var(χ²);
    y = X*β⁰ + u;
    βhat[j, 1:3] = X\y;
    uhat = uh(y, X, βhat[j, 1:3])
    σ²hat[j, 1:3] = diag(Sigmahat(X, uhat))    
end


The output of the following code is a matrix, βhat, and a matrix σ²hat. Each row of σhat contains the diagonal elements of $\hat{\Sigma}$ relative to the sample drawn corresponding to the given row. This means that we can standardize βhat and then compare the resulting histogram with that of a standard normal. 

In [ ]:
z = √n*(βhat.-β⁰')./√σ²hat;
Plots.histogram(z, normalize = true, layout = 3)
Plots.plot!(Normal(0,1), subplot = 1)
Plots.plot!(Normal(0,1), subplot = 2)
Plots.plot!(Normal(0,1), subplot = 3)

We should have expected somewhat big divergences between the densities of  $\hat{\beta}_0$, $\hat{\beta}_1$, and $\hat{\beta}_2$  and those of $N(0,1)$: we are using the asymptotic approximation when the actual sample size is $n=25$.

Let's do the same exercise, but with $n=200$. 

In [ ]:
n = 200;    ## <- one hundred observations
simulations = 100_000;
βhat  = Array(Float64, simulations, 3);
σ²hat  = Array(Float64, simulations, 3);
for j = 1:simulations
    X = [ones(n) rand(N, n) rand(N, n)];
    u = (rand(χ², n)-mean(χ²))/√var(χ²);
    y = X*β⁰ + u;
    βhat[j, 1:3] = X\y;
    uhat = uh(y, X, βhat[j, 1:3])
    σ²hat[j, 1:3] = diag(Sigmahat(X, uhat))    
end
z = √n*(βhat.-β⁰')./√σ²hat;


In [ ]:
Plots.histogram(z, normalize = true, layout = 3)
Plots.plot!(Normal(0,1), subplot = 1)
Plots.plot!(Normal(0,1), subplot = 2)
Plots.plot!(Normal(0,1), subplot = 3)

Now the densitites of $\hat{\beta}_0$, $\hat{\beta}_1$, and $\hat{\beta}_2$ are much closer to those of a standard normal. 

## Case 2: $E[x_i'u_i]=0$, but $E[u_i|x_i]\neq 0$
We now generate the data in such a way that $E[x_i'u_i]=0$ but $u_i$ and $x_i$ are mean dependent. In particular, the data are generated as follows
$$
y_i = x_i\beta^0 + u_i
$$
where $u_i = x_{i2}^2 - 1 + \epsilon_i$, $\epsilon_i \sim N(0,1)$, $x_{i2} \sim N(0,1)$. 
It should be clear that $E[u_i|x_i] \neq 0$ because
$$
E[u_i|x_i] = E[x_{i2}^2-1+\epsilon_i|x_i] = x_{i2}^2 -1 + E[\epsilon_i|x_i] = x_{i2}^2 - 1,
$$
but since $E[x_{i}'u_{i}] = \pmatrix{0 \\ Ex_{i2}^3 \\ 0}$ which is zero because the third moment of a normal is zero. 
The partial effect is given by
$$
\frac{\partial E[y_i|x_i]}{\partial x_{i2}} = \beta^0_{2} + 2x_{i2},
$$
which as expected is not equal to $\beta^0_{2}$.



In [ ]:
n = 25
simulations = 100_000
βhat  = Array(Float64, simulations, 3);
σ²hat  = Array(Float64, simulations, 3);
for j = 1:simulations
    X = [ones(n) rand(N, n) rand(N, n)];
    u = X[:,2].^2 + rand(N,n)-1;
    y = X*β⁰ + u;
    βhat[j, 1:3] = X\y;
    uhat = uh(y, X, βhat[j, 1:3])
    σ²hat[j, 1:3] = diag(Sigmahat(X, uhat))    
end


In [ ]:
mean(βhat,1)

This is a surprise! It seems the $\hat{\beta}_0$, $\hat{\beta}_1$ and $\hat{\beta}_2$ are _nearly_ unbiased. The are consistent, that is as $n\to\infty$ the difference between $\hat{\beta}$ and $\beta^0$ gets smaller and smaller. 

We can recover $\hat{\beta}_1$ and $\hat{\beta}_2$ very well, but they are useless for estimating the partial effect of $x_{i2}$ on the conditional expectation. 

## Case 3: $E[x_i'u_i]\neq0$ and $E[u_i|x_i]\neq 0$
We now generate the data in such a way that both assumptions are violated.  In particular, the data are generated as follows
$$
y_i = x_i\beta^0 + u_i
$$
where $u_i = x_{i2}^3 + \epsilon_i$, $\epsilon_i \sim N(0,1)$, $x_{i2} \sim N(0,1)$. 
It should be clear that $E[u_i|x_i] \neq 0$ because
$$
E[u_i|x_i] = E[x_{i2}^3+\epsilon_i|x_i] = x_{i2}^2 + E[\epsilon_i|x_i] = x_{i2}^3,
$$
but also $E[x_{i}'u_{i}] = \pmatrix{0 \\ Ex_{i2}^4 \\ 0}$ which is not zero because the fourt moment of a normal is not zero. 
The partial effect is given by
$$
\frac{\partial E[y_i|x_i]}{\partial x_{i2}} = \beta^0_{2} + 3x_{i2}^2.
$$



In [ ]:
n = 25
simulations = 100_000
βhat  = Array(Float64, simulations, 3);
σ²hat  = Array(Float64, simulations, 3);
for j = 1:simulations
    X = [ones(n) rand(N, n) rand(N, n)];
    u = X[:,2].^3 + rand(N,n);
    y = X*β⁰ + u;
    βhat[j, 1:3] = X\y;
    uhat = uh(y, X, βhat[j, 1:3])
    σ²hat[j, 1:3] = diag(Sigmahat(X, uhat))    
end

In [ ]:
mean(βhat,1)

In this case, $\hat{\beta}$ is estimating the linear projection that is given by $\beta = (1,3,0.8)$. The linear projection is not equal to $\beta^0$ and not equal to the partial effect. 